Speech Tokenizer Codec: https://github.com/ZhangXInFD/SpeechTokenizer

In [ ]:
#Load Model
from speechtokenizer import SpeechTokenizer

config_path = r'C:\Users\bolli\Desktop\Lekha\AudioDeepfake\models\config.json'
ckpt_path = r'C:\Users\bolli\Desktop\Lekha\AudioDeepfake\models\SpeechTokenizer.pt'
model = SpeechTokenizer.load_from_checkpoint(config_path, ckpt_path)
model.eval()

SpeechTokenizer(
  (encoder): SEANetEncoder(
    (model): Sequential(
      (0): SConv1d(
        (conv): NormConv1d(
          (conv): Conv1d(1, 64, kernel_size=(7,), stride=(1,))
          (norm): Identity()
        )
      )
      (1): SEANetResnetBlock(
        (block): Sequential(
          (0): ELU(alpha=1.0)
          (1): SConv1d(
            (conv): NormConv1d(
              (conv): Conv1d(64, 32, kernel_size=(3,), stride=(1,))
              (norm): Identity()
            )
          )
          (2): ELU(alpha=1.0)
          (3): SConv1d(
            (conv): NormConv1d(
              (conv): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
              (norm): Identity()
            )
          )
        )
        (shortcut): SConv1d(
          (conv): NormConv1d(
            (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
            (norm): Identity()
          )
        )
      )
      (2): ELU(alpha=1.0)
      (3): SConv1d(
        (conv): NormConv1d(
          (conv):

In [16]:
#Extracting discrete representations
import torchaudio
import torch

# Load and pre-process speech waveform
wav, sr = torchaudio.load(r'C:\Users\bolli\Desktop\Lekha\AudioDeepfake\models\small.wav')

print(wav.shape)


torch.Size([2, 5161984])


In [15]:
# monophonic checking
if wav.shape[0] > 1:
    wav = wav[:1,:]

if sr != model.sample_rate:
    wav = torchaudio.functional.resample(wav, sr, model.sample_rate)

wav = wav.unsqueeze(0)

# Extract discrete codes from SpeechTokenizer
with torch.no_grad():
    codes = model.encode(wav) # codes: (n_q, B, T)

RVQ_1 = codes[:1, :, :] # Contain content info, can be considered as semantic tokens
RVQ_supplement = codes[1:, :, :] # Contain timbre info, complete info lost by the first quantizer